# Convert VIAI Cosmetic Defect Dataset to Vertex AI Segmentation
Contributor: michaelmenzel@google.com

In [ ]:
#@title Parameters & Authenticate
ANNOTATION_FILE = "gs://viai-demo-data-us-central1/cosmetic-test-data-public/data_with_polygon.jsonl" # @param {type:"string"}
TARGET_BUCKET = "gs://viai-demo-data-us-central1/cosmetic-test-data-public/" # @param {type:"string"}



import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

In [ ]:
#@title Load Annotations
!gsutil cp $ANNOTATION_FILE .

import json
import os

with open(ANNOTATION_FILE.split(os.sep)[-1], 'r') as af:
  data = [json.loads(line) for line in af]

Copying gs://viai-demo-data-us-central1/cosmetic-test-data-public/data_with_polygon.jsonl...
/ [1 files][ 46.9 KiB/ 46.9 KiB]                                                
Operation completed over 1 objects/46.9 KiB.                                     


In [ ]:
#@title Convert Annotations

from tqdm import tqdm

vai_annotations = []

for item in tqdm(data):
  vai_annotations.append(
    {
        'imageGcsUri': item['image_gcs_uri'],
        'polygonAnnotations': [{
            "displayName": ann['annotation_spec'],
            "vertexes": ann['vi_bounding_poly']['vertex']
          } for ann in item['vi_annotations']]
    })

vai_annotations

100%|██████████| 35/35 [00:00<00:00, 30299.41it/s]


[{'imageGcsUri': 'gs://visual-inspection-demo-datasets-us-central1/cosmetic-test-data-public/public_original/IMG_0765.png',
  'polygonAnnotations': []},
 {'imageGcsUri': 'gs://visual-inspection-demo-datasets-us-central1/cosmetic-test-data-public/public_original/IMG_07656.png',
  'polygonAnnotations': [{'displayName': 'dent',
    'vertexes': [{'x': 0.8509615384615384, 'y': 0.6101485148514851},
     {'x': 0.8533653846153846, 'y': 0.6002475247524752},
     {'x': 0.8545673076923077, 'y': 0.6064356435643564}]},
   {'displayName': 'dent',
    'vertexes': [{'x': 0.8485576923076923, 'y': 0.6225247524752475},
     {'x': 0.8509615384615384, 'y': 0.6126237623762376},
     {'x': 0.8521634615384616, 'y': 0.6188118811881188}]},
   {'displayName': 'scratch',
    'vertexes': [{'x': 0.3173076923076923, 'y': 0.5482673267326733},
     {'x': 0.30288461538461536, 'y': 0.5482673267326733},
     {'x': 0.29447115384615385, 'y': 0.5420792079207921},
     {'x': 0.31971153846153844, 'y': 0.5408415841584159}]}]},

In [ ]:
#@title Store Annotations
with open('vertexai_segmentation_polygon.jsonl', 'w') as of:
  for l in vai_annotations:
    json.dump(l, of)
    of.write('\n')

!gsutil cp 'vertexai_segmentation_polygon.jsonl' $TARGET_BUCKET

Copying file://vertexai_segmentation_polygon.jsonl [Content-Type=application/octet-stream]...
/ [1 files][ 40.3 KiB/ 40.3 KiB]                                                
Operation completed over 1 objects/40.3 KiB.                                     
